SPLITTING TRAIN TEST FUNCTION 

In [6]:
from fastapi import FastAPI 
import pandas as pd 
from sklearn.model_selection import train_test_split

def split_data(csv_file, test_size=0.2):
  csv_file= ('/Users/miguelopez/Desktop/API/data/processed/df_final.csv')
  df = pd.read_csv(csv_file)
  
  X = df.drop('CarbonEmission', axis=1) 
  y = df['CarbonEmission']
  
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
  
  X_train.to_csv('X_train.csv', index=False)
  X_test.to_csv('X_test.csv', index=False)
  y_train.to_csv('y_train.csv', index=False) 
  y_test.to_csv('y_test.csv', index=False)

  print(f"Split data into train and test with {test_size*100}% in test set")


In [2]:
split_data ("data.csv") 

Split data into train and test with 20.0% in test set


RETURN PREDICTION 

In [3]:
# @app.get("/v1/train")
def train_api():

  # Load data
  df = pd.read_csv("data.csv")  
  X = df.drop('target', axis=1)
  y = df['target']

  # Split data
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

  # Train model
  model = LogisticRegression()
  model.fit(X_train, y_train)

  # Make predictions
  y_pred = model.predict(X_test)

  # Add predictions to test DataFrame
  X_test['predictions'] = y_pred

  return X_test.to_dict()


all in function 

In [4]:
# @app.get("/v1/train_and_predict")
def train_and_predict():

  # Load data
  df = pd.read_csv("/Users/miguelopez/Desktop/API/data/processed/df_encoded.csv")

  # Split data and save CSV files
  X = df.drop('target', axis=1)
  y = df['target']

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
  
  X_train.to_csv('X_train.csv', index=False)
  X_test.to_csv('X_test.csv', index=False)
  y_train.to_csv('y_train.csv', index=False)
  y_test.to_csv('y_test.csv', index=False)

  # Train model
  model = LogisticRegression()
  model.fit(X_train, y_train)  

  # Make predictions
  y_pred = model.predict(X_test)

  # Add predictions to test DataFrame
  X_test['predictions'] = y_pred

  return X_test.to_dict()


In [14]:
# @app.get("/v1/train_and_predict")
def train_and_predict():

  # Load data
  df = pd.read_csv("/Users/miguelopez/Desktop/API/data/processed/df_final.csv")

  # Split data and save CSV files
  X = df.drop('CarbonEmission', axis=1)
  y = df['CarbonEmission']

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
  
  X_train.to_csv('X_train.csv', index=False)
  X_test.to_csv('X_test.csv', index=False)
  y_train.to_csv('y_train.csv', index=False)
  y_test.to_csv('y_test.csv', index=False)

  # Train model
  model = DecisionTreeRegressor()               
  model.fit(X_train, y_train)  

  # Save model
  # model_path = '/Users/miguelopez/Desktop/API/model/model.pkl' 
  with open(model.pkl, 'wb') as f:
    pickle.dump(model, f)
  

  # Make predictions
  y_pred = model.predict(X_test)

  # Add predictions to test DataFrame
  X_test['predictions'] = y_pred

  return X_test.to_dict()


metrics 

In [18]:
import json
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# @app.get("/v2/metrics")
def metrics_api():

  # Load model
  with open('model.pkl', 'rb') as f:
    model = pickle.load(f)  

  # Load test data
  X_test = pd.read_csv('/Users/miguelopez/Desktop/API/X_test.csv')
  y_test = pd.read_csv('//Users/miguelopez/Desktop/API/y_test.csv')

  # Make predictions
  y_pred = model.predict(X_test)
  
  # Calculate metrics
  accuracy = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred)
  recall = recall_score(y_test, y_pred)
  f1 = f1_score(y_test, y_pred)
  auc = roc_auc_score(y_test, y_pred)

  metrics = {
    "accuracy": accuracy,
    "precision": precision, 
    "recall": recall,
    "f1": f1,
    "auc": auc
  }

  return json.dumps(metrics)


Metrics cont

In [ ]:
def metrics():
    model = pickle.load(open("model.pkl", "rb"))
    
    X_test = pd.read_csv("/Users/miguelopez/Desktop/API/data/processed/X_test.csv")
    y_test = pd.read_csv("/Users/miguelopez/Desktop/API/data/processed/y_test.csv")  
    
    y_pred = model.predict(X_test)
    R2 = r2_score(y_test, y_pred)
    MSE = mean_squared_error(y_test, y_pred)    
    MAE = mean_absolute_error(y_test, y_pred)   
    
    metrics = {"R2_Score": R2, 
               "MSE": MSE,
               "MAE": MAE,}
    
    return metrics




Docker notes 


In [ ]:
# Use an official Python runtime as the base image
FROM python:3.11.4

# Set the working directory in the container
WORKDIR /app

# Copy the requirements file to the container
COPY requirements.txt .

# Install dependencies
RUN pip install --no-cache-dir -r requirements.txt

# Copy the application code to the container
COPY . .

# Expose port for the API
EXPOSE 8000 

# Define environment variable for API
ENV API_PORT=8000

# Run the API when the container launches
CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "$API_PORT"]


In [ ]:
#  build into an image 
docker build -t myapi .
#  run container /access http://localhost:8000.
docker run -p 8000:8000 myapi
